In [9]:
import cv2
import mediapipe as mp
import time
import numpy as np
#import math
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()

mpDraw = mp.solutions.drawing_utils

In [10]:
pTime = 0
cTime = 0
x_pos = 0
y_pos = 0
sep = 0

finger_positions = np.zeros(shape=(22,3))
print(finger_positions[0])

[0. 0. 0.]


In [11]:
class PointCoordinates:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def middle_point(self, other):
        S_O_X_Middle = (self.x + other.x) / 2
        S_O_Y_Middle = (self.y + other.y) / 2
        return(S_O_X_Middle, S_O_Y_Middle)
        
    def point_seperation(self, other):
        S_O_X_Separation = self.x - other.x
        S_O_Y_Separation = self.y - other.y
        S_O_Separation = (((S_O_X_Separation ** 2) + (S_O_Y_Separation ** 2))) ** 0.5
        return(S_O_Separation)
finger_tops = [4, 8, 12, 16, 20]
finger_bottoms = [2, 6, 10, 14, 18]

In [12]:
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    for finger in range(0, len(finger_tops)):
        finger_1 = finger_positions[finger_tops[finger]][0], finger_positions[finger_tops[finger]][1]
        finger_2 = finger_positions[finger_bottoms[finger]][0], finger_positions[finger_bottoms[finger]][1]
        
        point_1 = PointCoordinates(finger_1[0], finger_1[1])
        point_2 = PointCoordinates(finger_2[0], finger_2[1])
        
        cv2.putText(img, f"({x_pos}, {y_pos}), Sep = ({sep:.2f})", (int(x_pos), int(y_pos)), cv2.FONT_HERSHEY_PLAIN, 1,
            (255, 0, 255), 1) 

        cv2.line(img, (int(point_1.x), int(point_1.y)), (int(point_2.x), int(point_2.y)), (0, 255, 0), thickness=2)
        x_pos, y_pos = PointCoordinates.middle_point(point_1, point_2)
        sep = PointCoordinates.point_seperation(point_1, point_2)
        if sep < 70:
            cv2.circle(img, (int(x_pos), int(y_pos)), 15, (255, 0, 0), cv2.FILLED)    
    


    
    
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            
            #print(temp_data_gathering)
            for id, lm in enumerate(handLms.landmark):
                # Goes through the individual fingers?
                
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                  #print(id, cx, cy
                  
                for point in range(0,22):
                    if point == id:
                        finger_positions[id][0] = cx
                        finger_positions[id][1] = cy
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 255), 3)
    #cv2.putText(img, f"Ball Coords (x,y,z): ({x_sep}, {int(finger_positions[0][1])}", (10, 450), cv2.FONT_HERSHEY_PLAIN, 1,
    #           (255, 0, 255), 1)
    
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)

KeyboardInterrupt: 